In [25]:
import pandas as pd
import tensorflow as tf
import re
ROOT_PATH = 'C:/Users/Hassan Raza/Desktop/Whatsapp_chat_RNN'
dataset = pd.read_csv(ROOT_PATH + '/Whatsapp_chat.csv', usecols=[0,1])

In [26]:
#dataset = dataset[dataset['Mode'].isin(['Positive', 'Negative'])]
#df4 = df[(df['count'].isin(['2','7']))
dataset["Mode"].value_counts()

Neutral     8929
Positive    6013
Negative    5287
Name: Mode, dtype: int64

In [33]:
corpus=[]
stopwords=['ai', 'ayi', 'hy', 'hai', 'main', 'ki', 'tha', 'koi', 'ko', 'sy', 'woh', 'bhi', 'aur', 'wo', 'yeh', 'rha', 'hota', 'ho', 'ga', 'ka', 'le', 'lye', 'kr', 'kar', 'lye', 'liye', 'hotay', 'waisay', 'gya', 'gaya', 'kch', 'ab', 'thy', 'thay', 'houn', 'hain', 'han', 'to', 'is', 'hi', 'jo', 'kya', 'thi', 'se', 'pe', 'phr', 'wala', 'waisay', 'us', 'na', 'ny', 'hun', 'rha', 'raha', 'ja', 'rahay', 'abi', 'uski', 'ne', 'haan', 'acha', 'nai', 'sent', 'photo', 'you', 'kafi', 'gai', 'rhy', 'kuch', 'jata', 'aye', 'ya', 'dono', 'hoa', 'aese', 'de', 'wohi', 'jati', 'jb', 'krta', 'lg', 'rahi', 'hui', 'karna', 'krna', 'gi', 'hova', 'yehi', 'jana', 'jye', 'chal', 'mil', 'tu', 'hum', 'par', 'hay', 'kis', 'sb', 'gy', 'dain', 'krny', 'tou']
for i in range(dataset.shape[0]):
    message = re.sub('[^a-zA-Z]',' ',str(dataset.iloc[:,0].values[i]))
    message = message.lower()
    message = message.split()
    message = [word for word in message if not word in stopwords]
    message = ' '.join(message)
    corpus.append(message)

In [34]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, split=" ")
tokenizer.fit_on_texts(corpus)

X = tokenizer.texts_to_sequences(corpus)
X = pad_sequences(X) # padding our text vector so they all have the same length (maximum lenth sub_list in list array)
len(X[0])

187

In [35]:
def FindMaxLength(lst):
	maxList = min(lst, key = lambda i: len(i))
	maxLength = len(maxList)
	
	return maxList, maxLength

#print(FindMaxLength(X))

In [36]:
model = Sequential()
model.add(Embedding(5000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [37]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 187, 256)          1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 187, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 187, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 2,331,395
Trainable params: 2,331,395
Non-trainable params: 0
_________________________________________________________________


In [47]:
y = pd.get_dummies(dataset['Mode']).values
[print(dataset['Mode'][i], y[i]) for i in range(12000,12001)]

# Positive [0 0 1]
# Neutral [0 1 0]
# Negative [1 0 0]

Negative [1 0 0]


[None]

In [48]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [51]:
batch_size = 128
epochs = 5

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/5
16183/16183 [==============================] - 888s 55ms/step - loss: 0.8793 - acc: 0.6076
Epoch 2/5
16183/16183 [==============================] - 1141s 71ms/step - loss: 0.6916 - acc: 0.7136
Epoch 3/5
16183/16183 [==============================] - 1118s 69ms/step - loss: 0.6137 - acc: 0.7496
Epoch 4/5
16183/16183 [==============================] - 1357s 84ms/step - loss: 0.5582 - acc: 0.7777
Epoch 5/5
16183/16183 [==============================] - 1388s 86ms/step - loss: 0.5099 - acc: 0.7980


In [52]:
model.save('sentiment_analysis.h5')

In [53]:
predictions = model.predict(X_test)

[print(dataset['Message'][i], predictions[i], y_test[i]) for i in range(5, 10)]

Wah kya baat likhi [0.64506716 0.34856185 0.00637098] [0 1 0]
Wha Itni sari khubiya [0.60560066 0.37095395 0.0234454 ] [0 1 0]
Itni khubiya [0.13057727 0.8137635  0.05565921] [0 1 0]
Ya allah rehm farma hm sab pe or zalimo ko hidayat de ameen [0.06729884 0.56694263 0.36575857] [1 0 0]
Please Everyone AllAh S.w.T ka naam hAmesha Bary Lawzo main Likha kary Wo he Zaat sUb say Bari Hey [0.31156573 0.01841947 0.6700148 ] [0 0 1]


[None, None, None, None, None]

In [54]:
pos_count, neu_count, neg_count = 0, 0, 0
real_pos, real_neu, real_neg = 0, 0, 0
for i, prediction in enumerate(predictions):
    if np.argmax(prediction)==2:
        pos_count += 1
    elif np.argmax(prediction)==1:
        neu_count += 1
    else:
        neg_count += 1
    
    if np.argmax(y_test[i])==2:
        real_pos += 1
    elif np.argmax(y_test[i])==1:    
        real_neu += 1
    else:
        real_neg +=1

print('Positive predictions:', pos_count)
print('Neutral predictions:', neu_count)
print('Negative predictions:', neg_count)
print('Real positive:', real_pos)
print('Real neutral:', real_neu)
print('Real negative:', real_neg)

Positive predictions: 1120
Neutral predictions: 1835
Negative predictions: 1091
Real positive: 1182
Real neutral: 1832
Real negative: 1032
